# Trabalho Prático 3  #
#### André Freitas PG54707 ####
#### Bruna Macieira PG54467 ####

## Exercício 1

No capítulo 5 dos apontamentos é descrito o chamado Hidden Number Problem. No capítulo 8 dos apontamentos é discutida um artigo de  Nguyen & Shparlinsk , onde se propõem reduções do HNP a problemas difíceis em reticulados. Neste trabalho pretende-se construir, com a ajuda do Sagemath, uma implementação da solução discutida nos apontamentos para resolver o HNP com soluções aproximadas dos problemas em reticulados.

In [728]:
%pip install sagemath-standard
from sage.all import *
import copy

  Using cached importlib_resources-6.4.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached networkx-3.1-py3-none-any.whl.metadata (5.3 kB)
  Using cached scipy-1.10.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (58 kB)
  Using cached fpylll-0.5.9.tar.gz (1.1 MB)
  Preparing metadata (setup.py) ... done
  Using cached ptyprocess-0.5.1-py2.py3-none-any.whl.metadata (1.3 kB)
Using cached ptyprocess-0.5.1-py2.py3-none-any.whl (12 kB)
Using cached importlib_resources-6.4.0-py3-none-any.whl (38 kB)
Using cached networkx-3.1-py3-none-any.whl (2.1 MB)
Using cached scipy-1.10.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.1 MB)
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [67 lines of output]
      INFO: Disabling color, you really want to install colorlog.
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller

Etapa 1: Construção da matriz 𝐺.

In [729]:
def build_G_matrix(p, x_values, A, u_values, M, B):
    n = len(x_values)
    m = n + 2
    G = Matrix(QQ, m, m)

    # Preenchendo os elementos de G
    for i in range(m):
        for j in range(m):
            if i < n and j < n:
                G[i, j] = p if i == j else 0
            elif i == n:
                G[i, j] = x_values[j] if j < n else A if j == n else 0
            elif i == n + 1:
                G[i, j] = -B * u_values[j] if j < n else 0 if j == n else M
    print("G:", G)

    return G

Etapa 2: Construção do vetor alvo 𝑦.

In [730]:
def build_y_vector(u_values, B, M):
    n = len(u_values)
    y = [B * u for u in u_values] + [0, M]  # Construindo o vetor y

    return vector(y)

Etapa 3: Resolução do Problema do Menor Vetor em uma Reticulado (BDD).

In [731]:
def solve_BDD(G, y, B):
    # Ensure G and y have compatible dimensions
    if len(G[0]) == len(y):
        # Convert y to a SageMath vector
        y_vector = vector(y)

        # Construct the lattice and reduce the basis using the LLL algorithm
        L = Matrix(G)
        n = L.nrows()
        k = 2
        L_star = copy.deepcopy(L)  # Use deepcopy to create a copy of the matrix
        while k <= n:
            for j in range(1, k):
                mu_kj = L[k-1] * L_star[j-1] / (L_star[j-1] * L_star[j-1])
                L[k-1] = L[k-1] - mu_kj * L_star[j-1]  # Size Reduction
                print(f"Size reduction: L[{k-1}] = {L[k-1]}")  # Debugging print statement
            if L_star[k-1] * L_star[k-1] >= (3/4 - (L[k-1] * L_star[k-2] / (L_star[k-2] * L_star[k-2]))**2) * L_star[k-2] * L_star[k-2]:  # Lovász Condition
                k += 1
                print(f"Increasing k to {k}")  # Debugging print statement
            else:
                L[k-1], L[k-2] = L[k-2], L[k-1]  # Swap Step
                k = max(2, k - 1)
                print(f"Swapping vectors and setting k to {k}")  # Debugging print statement

        # Calculate the closest point to y in the lattice
        closest_point = L * y_vector
        print("closest point:", closest_point)

        # Round the closest point
        rounded_point = vector(ZZ, [round(coord) for coord in closest_point])
        print("B:", B)
        print("rounded point:", rounded_point)
        print("y_vector:", y_vector)
        print("distance:", vector(rounded_point - y_vector).norm())

        # Check if the distance is less than B
        if vector(rounded_point - y_vector).norm() <= B:
            return rounded_point
        else:
            print("The closest point found is not within the expected distance")
            return None
    else:
        print("G and y have incompatible dimensions")
        return None

Etapa 4: Recuperação do segredo 𝑠 a partir do vetor solução.

In [732]:
def recover_secret(reduced_matrix, lambda_value):
    # Extraindo a última componente do vetor solução
    e_n_plus_1 = reduced_matrix[-1][-2]

    # Arredondando para obter o valor de s
    s = round(lambda_value * e_n_plus_1)

    return s

Etapa 5: Avaliação da precisão da solução.

In [733]:
# Função para avaliar a precisão da solução
def evaluate_solution(y, x_values, s, p, B):
    # Calculando o vetor obtido a partir do segredo recuperado
    y_obtained = vector([s * xi % p for xi in x_values] + [0, 0])

    # Calculando a distância entre y e y_obtained
    distance = vector(y - y_obtained).norm()

    # Verificando se a distância é menor ou igual a B
    if distance <= B:
        return True, distance
    else:
        return False, distance

Definir os parâmetros a usar para resolver

In [734]:
k = 2  # Definindo o valor de k
lambda_value = 2**k  # Calculando o valor de lambda

# Parâmetros do problema
p = next_prime(101)  # Primo p
x_values = [2, 1, 1, 2, 3]  # Valores xi
u_values = [1, 1, 2, 2, 3]  # Valores ui
A = 1/lambda_value  # Valor A é 1/lambda
B = p/lambda_value # Parâmetro B é p/lambda
M = 1001  # Valor M

In [735]:
# Construindo a matriz G
G = build_G_matrix(p, x_values, A, u_values, M, B)

G: [   103      0      0      0      0      0      0]
[     0    103      0      0      0      0      0]
[     0      0    103      0      0      0      0]
[     0      0      0    103      0      0      0]
[     0      0      0      0    103      0      0]
[     2      1      1      2      3    1/4      0]
[-103/4 -103/4 -103/2 -103/2 -309/4      0   1001]


In [736]:
# Construindo o vetor alvo y
y = build_y_vector(u_values, B, M)

In [737]:
# Resolvendo o BDD
solution = solve_BDD(G, y, B)
print("Solution:", solution)

G and y have incompatible dimensions
Solution: None


In [738]:
# Verificando se a solução foi encontrada
if solution:
    # Recuperando o segredo
    s = recover_secret(solution, A)

    # Avaliando a precisão da solução
    is_within_limit, distance = evaluate_solution(y, x_values, s, p, B)

    # Exibindo resultados
    if is_within_limit:
        print("Solução encontrada dentro do limite B.")
        print("Segredo recuperado:", s)
        print("Distância entre y e y_obtained:", distance)
    else:
        print("Solução encontrada, mas está fora do limite B.")
        print("Segredo recuperado:", s)
        print("Distância entre y e y_obtained:", distance)
else:
    print("Não foi possível encontrar uma solução dentro do limite B.")

Não foi possível encontrar uma solução dentro do limite B.
